In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

# Week 1

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
#X_init = np.array([...])  # 30x6 array
#y_init = np.array([...])  # 30 outputs
X_init = datain
y_init = dataout

y_best = y_init.max()  # best performance so far

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimization in scipy

# --- Optimize acquisition function ---
bounds = [(0,1)]*6
best_x = None
best_ei = float('inf')

for _ in range(20):  # multiple random starts
    x0 = np.random.rand(6)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


# Week 2

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
#X_init = np.array([...])  # 30x6 array
#y_init = np.array([...])  # 30 outputs
X_init = datain
y_init = dataout

X_new = np.array([[0.067502, 0.634745, 0.770678, 0.925945, 0.431622, 0.542899]])
y_new = np.array([0.03701472059292424])

X_init = np.vstack([X_init, X_new])
y_init = np.append(y_init, y_new)

y_best = y_init.max()  # best performance so far

print(X_init)
print(y_init)
print(y_best)

In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)
y_best = y_init.max()  # still the best observed performance


In [ ]:
def expected_improvement(x, gp, y_best, xi=0.05):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0:
        return 0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimize


In [ ]:
bounds = [(0, 1)] * 6
best_x = None
best_ei = float('inf')

for _ in range(20):
    x0 = np.random.rand(6)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


# Week 3

In [ ]:
this new sample is not an improvement — it’s far below the best observed score.
(a) Increase exploration slightly
You’ve now had two consecutive low-performing samples after your initial batch of 30.
(b) Use more random restarts or a global search for EI
If you’re repeatedly landing in low-output zones, your acquisition optimization might be getting trapped in local minima.

The overall procedure for selecting the next point stays the same — refit the GP and maximize Expected Improvement — but given two recent low outputs, you should now:
Increase exploration slightly (e.g. raise xi in EI),
Use more restarts or random sampling in optimizing EI,
Optionally test an alternative acquisition like UCB.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Load initial data ---
X_init = np.load('initial_inputs.npy')   # 30x6
y_init = np.load('initial_outputs.npy')  # 30 outputs

# --- Add new observations ---
new_X = np.array([
    [0.067502, 0.634745, 0.770678, 0.925945, 0.431622, 0.542899],
    [0.400345, 0.617435, 0.564076, 0.028873, 0.730474, 0.787245]
])
new_y = np.array([0.03701472059292424, 0.21975396773107775])

X_init = np.vstack([X_init, new_X])
y_init = np.append(y_init, new_y)

# --- Update best observed performance ---
y_best = y_init.max()

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.05):  # increase xi for exploration
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimization

# --- Optimize acquisition function ---
bounds = [(0, 1)] * 6
best_x = None
best_ei = float('inf')

# Use more random restarts for robust optimization
n_restarts = 50
for _ in range(n_restarts):
    x0 = np.random.rand(6)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.05),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


In [ ]:
x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Next hyperparameter set to try:", x_next_6dp)

# Week 4

Current y_best = 1.3649683 (from the initial batch).
New point: [0.508267,0.997521,0.358960,0.745010,0.251294,0.856409] -> 0.018112
this new point is also far below the best observed performance, and in fact is even lower than the first low sample.
    
Current xi=0.05 is moderate. Given repeated poor outputs, you might raise it to xi=0.1 to more aggressively explore unknown regions.
This encourages EI to consider uncertain or far-away regions rather than small tweaks around known low-y zones.


The new output (0.018) reinforces that previously sampled regions are low-y.
The GP posterior now “punishes” these areas, so EI favors unexplored or uncertain regions.
Next point selection should refit the GP, increase xi to promote exploration
and use more optimizer restarts to better cover the 6D search space.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Load initial data ---
X_init = np.load('initial_inputs.npy')   # 30x6
y_init = np.load('initial_outputs.npy')  # 30 outputs

# 1. --- Add new observations ---
new_X = np.array([
    [0.067502, 0.634745, 0.770678, 0.925945, 0.431622, 0.542899],
    [0.400345, 0.617435, 0.564076, 0.028873, 0.730474, 0.787245],
    [0.508267, 0.997521, 0.358960, 0.745010, 0.251294, 0.856409]
])
new_y = np.array([0.03701472059292424, 0.21975396773107775, 0.018112125701738212])

X_init = np.vstack([X_init, new_X])
y_init = np.append(y_init, new_y)

# 2. Update best observed
y_best = y_init.max()

# 3. Refit GP
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.1):  # increase xi
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0:
        return 0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei

# 4. Maximize Expected Improvement with increased exploration
bounds = [(0, 1)]*6
best_x, best_ei = None, float('inf')
n_restarts = 100  # more random starts

for _ in range(n_restarts):
    x0 = np.random.rand(6)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.1),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)

In [ ]:
x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Rounded to:", x_next_6dp)

# Week 5

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Load initial data ---
X_init = np.load('initial_inputs.npy')   # 30x6
y_init = np.load('initial_outputs.npy')  # 30 outputs

# 1. --- Add new observations ---
new_X = np.array([
    [0.067502, 0.634745, 0.770678, 0.925945, 0.431622, 0.542899],
    [0.400345, 0.617435, 0.564076, 0.028873, 0.730474, 0.787245],
    [0.508267, 0.997521, 0.358960, 0.745010, 0.251294, 0.856409],
    [0.969578, 0.930527, 0.346056, 0.836240, 0.228505, 0.712374]
])
new_y = np.array([0.03701472059292424, 0.21975396773107775, 0.018112125701738212, 0.0034485098578360953])

X_init = np.vstack([X_init, new_X])
y_init = np.append(y_init, new_y)

x = [0.969578, 0.930527, 0.346056, 0.836240, 0.228505, 0.712374] → y = 0.00345
- is another sample with a value essentially zero compared to the current best (~1.365).
- It behaves like the other recent low-y samples:
- it will reduce uncertainty locally where it was sampled and depress the GP posterior mean there,
- but it does not challenge the dominant influence of the single high-y point we already have.
- It strengthens the evidence that high-performance regions are rare and not located near the recently sampled points.
- Exploration need: Multiple low-y hits in different parts of the 6-D space suggest the high-y region is sparse — the optimizer should explore more widely rather than continue making local tweaks around already-tested regions.

- The GP already knows that several regions produce low outputs
- EI now actively avoids them because uncertainty there is reduced.
- A large random pool encourages sampling from regions farther from known points, where variance remains high.
- This is a deliberately exploration-heavy move
- exactly what we want now that multiple recent samples have yielded extremely low values .

In [ ]:
import numpy as np
from scipy.stats import norm

# Refit GP
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

y_best = y_init.max()

def expected_improvement_batch(X, gp, y_best, xi=0.1):
    """
    Computes EI for a batch of candidate points X (shape: N x d).
    """
    mu, sigma = gp.predict(X, return_std=True)
    sigma = sigma.reshape(-1)
    
    # Avoid divide-by-zero
    sigma = np.maximum(sigma, 1e-12)

    improvement = mu - y_best - xi
    Z = improvement / sigma

    ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma < 1e-12] = 0.0  # no uncertainty -> no improvement

    return ei

# --- Generate large random candidate pool ---
num_candidates = 5000  # feel free to increase to 20k if compute is cheap
candidates = np.random.rand(num_candidates, 6)  # search space assumed [0,1]^6

# --- Evaluate EI for all candidates ---
ei_values = expected_improvement_batch(candidates, gp, y_best, xi=0.1)

# --- Select next point ---
best_idx = np.argmax(ei_values)
x_next = candidates[best_idx]

print("Exploration-first next suggestion (xi=0.1):")
print(x_next)


In [ ]:
x_next_6dp = np.round(x_next, 6)
print("Rounded to:", x_next_6dp)

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Load initial data ---
X_init = np.load('initial_inputs.npy')   # 30x6
y_init = np.load('initial_outputs.npy')  # 30 outputs

# 1. --- Add new observations ---
new_X = np.array([
    [0.067502, 0.634745, 0.770678, 0.925945, 0.431622, 0.542899],
    [0.400345, 0.617435, 0.564076, 0.028873, 0.730474, 0.787245],
    [0.508267, 0.997521, 0.358960, 0.745010, 0.251294, 0.856409],
    [0.969578, 0.930527, 0.346056, 0.836240, 0.228505, 0.712374],
    [0.120703, 0.031424, 0.856331, 0.160061, 0.069200, 0.239120]
])
new_y = np.array([0.03701472059292424, 0.21975396773107775, 0.018112125701738212, 0.0034485098578360953, 0.16586065795194388])

X_init = np.vstack([X_init, new_X])
y_init = np.append(y_init, new_y)


- This new sample x = [0.120703, 0.031424, 0.856331, 0.160061, 0.069200, 0.239120] → y = 0.16586
- is another low-to-mid value (well below the best ≈ 1.36497).
- It behaves like the other recent non-best samplesit will reduce uncertainty locally, depress the GP posterior mean in its neighborhood, and therefore make EI avoid that neighborhood in the next suggestion.
- It doesn’t challenge the dominant high-y point, but it adds useful negative evidence about where the maximum isn’t.

- Refit the GP including this new point, then run an exploration-first EI sweep with xi=0.1 over a large random candidate pool (5k–20k)
- and also keep one or two local-refinement candidates around the current best; choose the highest-EI candidate(s) to evaluate next.

In [ ]:
import numpy as np
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# Refit GP
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

y_best = y_init.max()

def expected_improvement_batch(X, gp, y_best, xi=0.1):
    """
    Compute EI for a batch of candidate points X (N x 6).
    """
    mu, sigma = gp.predict(X, return_std=True)
    sigma = np.maximum(sigma, 1e-12)

    improvement = mu - y_best - xi
    Z = improvement / sigma

    ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma < 1e-12] = 0.0

    return ei

# --- Exploration-first candidate sweep ---
num_candidates = 8000   # increase to 20,000 if fast enough
candidates = np.random.rand(num_candidates, 6)   # uniform search in [0,1]^6

# Evaluate EI on all candidates
ei_values = expected_improvement_batch(candidates, gp, y_best, xi=0.1)

# Select highest-EI point
best_idx = np.argmax(ei_values)
x_next = candidates[best_idx]

print("Next suggested hyperparameter point (exploration-first, xi=0.1):")
print(x_next)

In [ ]:
x_next_6dp = np.round(x_next, 6)
print("Rounded to:", x_next_6dp)

# Week 6
- Rounded to: [0.441479 0.75075  0.472307 0.601996 0.790739 0.013478]
- This is significantly higher than any previous observation — nearly double the previous maximum.
- EI will now prioritize regions around this new high-y point for local refinement.
- EI is likely to split between local exploitation around this new peak and exploration elsewhere where uncertainty remains high.
- This is exactly the situation where a dual strategy (explore globally, exploit locally) is beneficial.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Load initial data ---
X_init = np.load('initial_inputs.npy')   # 30x6
y_init = np.load('initial_outputs.npy')  # 30 outputs

# 1. --- Add new observations ---
new_X = np.array([
    [0.067502, 0.634745, 0.770678, 0.925945, 0.431622, 0.542899],
    [0.400345, 0.617435, 0.564076, 0.028873, 0.730474, 0.787245],
    [0.508267, 0.997521, 0.358960, 0.745010, 0.251294, 0.856409],
    [0.969578, 0.930527, 0.346056, 0.836240, 0.228505, 0.712374],
    [0.120703, 0.031424, 0.856331, 0.160061, 0.069200, 0.239120],
    [0.441479, 0.750750, 0.472307, 0.601996, 0.790739, 0.013478]
])
new_y = np.array([0.03701472059292424, 0.21975396773107775, 0.018112125701738212,
                  0.0034485098578360953, 0.16586065795194388, 2.5372438665513246])

X_init = np.vstack([X_init, new_X])
y_init = np.append(y_init, new_y)

# Refit GP
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)

In [ ]:
import numpy as np
from scipy.stats import norm

# --- Assumes GP is already fitted and y_best updated ---

gp.fit(X_init, y_init)
y_best = y_init.max()

def expected_improvement_batch(X, gp, y_best, xi=0.1):
    mu, sigma = gp.predict(X, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    improvement = mu - y_best - xi
    Z = improvement / sigma
    ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma < 1e-12] = 0.0
    return ei

# --- 1. Local exploitation around the new high-y point ---
new_peak = np.array([0.441479, 0.750750, 0.472307, 0.601996, 0.790739, 0.013478])
n_local = 200  # number of local candidates
perturbation = 0.05  # ±5% per dimension
local_candidates = new_peak + (np.random.rand(n_local, 6) - 0.5) * 2 * perturbation
local_candidates = np.clip(local_candidates, 0, 1)  # keep within bounds

ei_local = expected_improvement_batch(local_candidates, gp, y_best, xi=0.1)

# --- 2. Global exploration ---
n_global = 5000
global_candidates = np.random.rand(n_global, 6)
ei_global = expected_improvement_batch(global_candidates, gp, y_best, xi=0.1)

# --- 3. Combine and select next point ---
all_candidates = np.vstack([local_candidates, global_candidates])
all_ei = np.concatenate([ei_local, ei_global])

best_idx = np.argmax(all_ei)
x_next = all_candidates[best_idx]

print("Next candidate point to evaluate:")
print(x_next)

x_next_6dp = np.round(x_next, 6)
print("Rounded to:", x_next_6dp)


# Week 7
- x = [0.420520, 0.796570, 0.471162, 0.614847, 0.752107, 0.000000] → y = 2.756700494633134 is now the best observation so far (previous best ≈ 2.537).
- It is near the earlier high points, which strongly suggests a real high-performance basin has been located (not just a noisy outlier).
- This shifts the optimizer’s priorities: local refinement around this region becomes higher-value, while still keeping some global exploration.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Load initial data ---
X_init = np.load('initial_inputs.npy')   # 30x6
y_init = np.load('initial_outputs.npy')  # 30 outputs

# 1. --- Add new observations ---
new_X = np.array([
    [0.067502, 0.634745, 0.770678, 0.925945, 0.431622, 0.542899],
    [0.400345, 0.617435, 0.564076, 0.028873, 0.730474, 0.787245],
    [0.508267, 0.997521, 0.358960, 0.745010, 0.251294, 0.856409],
    [0.969578, 0.930527, 0.346056, 0.836240, 0.228505, 0.712374],
    [0.120703, 0.031424, 0.856331, 0.160061, 0.069200, 0.239120],
    [0.441479, 0.750750, 0.472307, 0.601996, 0.790739, 0.013478],
    [0.420520, 0.796570, 0.471162, 0.614847, 0.752107, 0.000000]
])
new_y = np.array([0.03701472059292424, 0.21975396773107775, 0.018112125701738212,
                  0.0034485098578360953, 0.16586065795194388, 2.5372438665513246,
                  2.756700494633134])

X_init = np.vstack([X_init, new_X])
y_init = np.append(y_init, new_y)

# Refit GP
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)


In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor

def dual_strategy_next_points(
    X, y, gp: GaussianProcessRegressor,
    k=10,
    n_global=20000,
    n_local=5000,
    xi_global=0.1,
    xi_local=0.01,
    trust_radius=0.20,
):
    """
    Implements:
      - aggressive exploration (xi=0.1)
      - local refinement near best region (xi=0.01)
    """

    dim = X.shape[1]

    # -------------------------------
    # 1. GLOBAL EXPLORATION
    # -------------------------------
    Xcand_global = np.random.rand(n_global, dim)

    mu_g, std_g = gp.predict(Xcand_global, return_std=True)
    ucb_g = mu_g + xi_global * std_g

    # Select top-k highest UCB
    global_idx = np.argsort(ucb_g)[-k:]
    global_candidates = Xcand_global[global_idx]

    # -------------------------------
    # 2. LOCAL REFINEMENT
    # -------------------------------
    # Use the current best y as the center
    best_idx = np.argmax(y)
    x_best = X[best_idx]

    # Uniform random samples in a ball
    # Using normal distribution normalized to radius
    Z = np.random.randn(n_local, dim)
    Z = Z / np.linalg.norm(Z, axis=1, keepdims=True)
    R = np.random.rand(n_local, 1) ** (1.0 / dim)  # proper ball distribution
    local_offsets = Z * R * trust_radius
    Xcand_local = np.clip(x_best + local_offsets, 0.0, 1.0)

    mu_l, std_l = gp.predict(Xcand_local, return_std=True)
    ucb_l = mu_l + xi_local * std_l

    # Select top-k strongest exploitation-weighted points
    local_idx = np.argsort(ucb_l)[-k:]
    local_candidates = Xcand_local[local_idx]

    return {
        "global_candidates": global_candidates,
        "local_candidates": local_candidates,
        "global_ucb": ucb_g[global_idx],
        "local_ucb": ucb_l[local_idx],
    }


# -----------------------
# Example usage:
# -----------------------
# Assuming you already updated X and y:
# gp.fit(X, y)
# results = dual_strategy_next_points(X, y, gp, k=10)
# print("Top global exploratory points:\n", results["global_candidates"])
# print("Top local refinement points:\n", results["local_candidates"])


In [ ]:
gp.fit(X_init, y_init)
results = dual_strategy_next_points(X_init, y_init, gp, k=10)
print("Top global exploratory points:\n", results["global_candidates"])
print("Top local refinement points:\n", results["local_candidates"])

#x_next_6dp = np.round(x_next, 6)
#print("Rounded to:", x_next_6dp)

In [ ]:
print("global_ucb:\n", results["global_ucb"])
print("local_ucb:\n", results["local_ucb"])

# Going with this
[0.49725918 0.80920918 0.41406537 0.67682857 0.78169744 0.00916812]

In [ ]:
sel = [0.49725918, 0.80920918, 0.41406537, 0.67682857, 0.78169744, 0.00916812]
x_next_6dp = np.round(sel, 6)
print("Rounded to:", x_next_6dp)

# Week 9
- This is very high (previously we saw 2.7567 as the top). 2.2299 is slightly lower than the absolute best (2.7567) but still much larger than most earlier samples.
- It confirms that a region in this neighborhood (roughly dims ~0.42–0.8 for several dims) contains high-y values — i.e., there is a promising basin of high performance.

- Compute EI

- this looks wrong

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Load initial data ---
X_init = np.load('initial_inputs.npy')   # 30x6
y_init = np.load('initial_outputs.npy')  # 30 outputs

# 1. --- Add new observations ---
new_X = np.array([
    [0.067502, 0.634745, 0.770678, 0.925945, 0.431622, 0.542899],
    [0.400345, 0.617435, 0.564076, 0.028873, 0.730474, 0.787245],
    [0.508267, 0.997521, 0.358960, 0.745010, 0.251294, 0.856409],
    [0.969578, 0.930527, 0.346056, 0.836240, 0.228505, 0.712374],
    [0.120703, 0.031424, 0.856331, 0.160061, 0.069200, 0.239120],
    [0.441479, 0.750750, 0.472307, 0.601996, 0.790739, 0.013478],
    [0.420520, 0.796570, 0.471162, 0.614847, 0.752107, 0.000000],
    [0.497259, 0.809209, 0.414065, 0.676829, 0.781697, 0.009168]
])
new_y = np.array([0.03701472059292424, 0.21975396773107775, 0.018112125701738212,
                  0.0034485098578360953, 0.16586065795194388, 2.5372438665513246,
                  2.756700494633134, 2.229887072011708])


X_init = np.vstack([X_init, new_X])
y_init = np.append(y_init, new_y)


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C, WhiteKernel
import numpy as np

# --- Rebuild GP with ARD ---
kernel = (
    C(1.0, (1e-3, 1e3)) *
    Matern(length_scale=np.ones(6), length_scale_bounds=(1e-3, 1e3), nu=2.5)
) + WhiteKernel(noise_level=1e-4, noise_level_bounds=(1e-6, 1e-2))

gp = GaussianProcessRegressor(
    kernel=kernel,
    normalize_y=True,
    alpha=1e-6,                 # numerical jitter
    n_restarts_optimizer=20,    # IMPORTANT
    random_state=42
)

gp.fit(X_init, y_init)
y_best = y_init.max()


In [ ]:
import numpy as np
from scipy.stats import norm

def expected_improvement(X, gp, y_best, xi=0.1):
    """
    Computes EI for a batch of candidate points X (N x d)
    X: array of candidates
    gp: already fitted GaussianProcessRegressor with ARD
    y_best: scalar, current best observed y
    xi: exploration-exploitation tradeoff
    """
    mu, sigma = gp.predict(X, return_std=True)
    sigma = np.maximum(sigma, 1e-12)  # avoid divide-by-zero

    improvement = mu - y_best - xi
    Z = improvement / sigma
    ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma < 1e-12] = 0.0
    return ei

def select_topk_ei_candidates(gp, y_best, num_candidates=5000, top_k=5, xi=0.1, dim=6):
    """
    Generates a large random candidate pool and selects top-k points by EI.
    """
    # Generate candidate pool
    candidates = np.random.rand(num_candidates, dim)

    # Compute EI
    ei_values = expected_improvement(candidates, gp, y_best, xi=xi)

    # Select top-k
    top_indices = np.argsort(-ei_values)[:top_k]  # descending
    top_candidates = candidates[top_indices]
    top_ei = ei_values[top_indices]

    return top_candidates, top_ei

# --- Usage example ---
# Assume gp is your refitted GP with ARD and y_best is updated
top_candidates, top_ei = select_topk_ei_candidates(gp, y_best, num_candidates=10000, top_k=5, xi=0.1)
print("Top-k EI candidates:\n", top_candidates)
print("Their EI values:\n", top_ei)


In [ ]:
import numpy as np
from scipy.stats import norm

def expected_improvement(X, gp, y_best, xi=0.1):
    """Batch EI computation"""
    mu, sigma = gp.predict(X, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    improvement = mu - y_best - xi
    Z = improvement / sigma
    ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma < 1e-12] = 0.0
    return ei

def select_topk_local_ei(gp, y_best, best_point, top_k=5, n_candidates=200, perturbation=0.05):
    """
    Generates local candidate points near best_point, evaluates EI, and returns top-k candidates.
    
    Parameters:
    - gp: fitted GP
    - y_best: current best output
    - best_point: 1D array of current best point
    - top_k: number of candidates to return
    - n_candidates: number of local candidates to sample
    - perturbation: maximum relative perturbation per dimension (0-1 scale)
    """
    dim = len(best_point)
    
    # Generate local candidates by adding small random perturbations
    local_candidates = best_point + (np.random.rand(n_candidates, dim) - 0.5) * 2 * perturbation
    local_candidates = np.clip(local_candidates, 0, 1)  # keep within bounds
    
    # Compute EI
    ei_values = expected_improvement(local_candidates, gp, y_best, xi=0.1)
    
    # Select top-k candidates
    top_indices = np.argsort(-ei_values)[:top_k]
    top_candidates = local_candidates[top_indices]
    top_ei = ei_values[top_indices]
    
    return top_candidates, top_ei

# --- Usage Example ---
# Assume gp is fitted, y_best is updated, and best_point is your current best-y point
best_point = np.array([0.420520, 0.796570, 0.471162, 0.614847, 0.752107, 0.000000])
top_candidates, top_ei = select_topk_local_ei(gp, y_best, best_point, top_k=5)
print("Top-k local EI candidates:\n", top_candidates)
print("Their EI values:\n", top_ei)


In [ ]:
x_next = [0.37371524, 0.83503778, 0.4466936, 0.60594758, 0.74148909, 0.0]
print("Next point (6 dp):", np.round(x_next, 6))